# Convolutional Neural Networks

## Project: Write an Algorithm for a Fruit Identification App 


---

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step3): Create a CNN to Classify Fruit types (from Scratch)
* [Step 2](#step4): Create a CNN to Classify Fruit types (using Transfer Learning)
* [Step 3](#step5): Write your Algorithm
* [Step 4](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets


In [23]:
import numpy as np
from glob import glob
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.autograd import Variable
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import os
#import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline    
import os
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  

# check if CUDA is available
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load filenames for human and dog images
fruit_files = np.array(glob("images/*/*/*"))

# print number of images in each dataset
print('There are %d total fruit images.' % len(fruit_files))

There are 76824 total fruit images.


---
<a id='step2'></a>
## Step 2: Detect Fruits

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect fruits in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [2]:

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:22<00:00, 24652780.22it/s]


Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as `'image/Training/*/*.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  The output should always be an integer between 0 and 999, inclusive.

Before writing the function, make sure that you take the time to learn  how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [3]:

def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''
    
    ## Load and pre-process an image from the given img_path
    ## Return the *index* of the predicted class for that image
    image = Image.open(img_path)
    
    # Define transformations for the image
    transformation = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Preprocess the image
    image_t = transformation(image).float()

    # Add extra dimension to treat as batch
    image_t = image_t.unsqueeze_(0)
    
    # Check appropriate device type
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    #Send to GPU if available
    image_t = image_t.to(device)  
    # Turn the input into a Variable
    image_model_input = Variable(image_t)
    
    #Prepare model
    VGG16.to(device).eval()
    output = VGG16(image_model_input)
    
    #Get values and index of max value
    values, index = torch.max(output, 1)
    return index.item()

### (IMPLEMENTATION) Write a Fruit Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), we will return the index of the ImageNet class from the function below.


In [4]:
### returns "True" if a fruit is detected in the image stored at img_path
def fruit_detector(img_path):
    ## TODO: Complete the function.
    index = VGG16_predict(img_path)

    return index

### (Test Observations) Results from pretrained model

Some fruit classes look to perform very well, but others are very poor, like Bananas, Lemons, and Oranges.  We will create a custom model training to try to improve our results.

In [5]:
# Test fruit detector function
# Not all the fruit classes are in the ImageNet dataset, pick a few known ones and evaluate

fruit_detector_fruit_count = 0

appleFiles = np.array(glob("/home/workspace/images/*/Apple Granny Smith/*"))
apple_files_short = fruit_files_short = appleFiles[:100]

for path in apple_files_short:
    # add bounding box to color image 
    resultindex = fruit_detector(path)
    if resultindex == 948:
        fruit_detector_fruit_count+=1

print("Percent of apples images detected as an apple: %d" % (fruit_detector_fruit_count))

fruit_detector_fruit_count = 0 

banana_files = np.array(glob("/home/workspace/images/*/Banana/*"))
banana_files_short = banana_files[:100]

for path in banana_files_short:
    resultindex = fruit_detector(path)
    if resultindex == 954:
        fruit_detector_fruit_count+=1

print("Percent of bananas images detected as a banana: %d" % (fruit_detector_fruit_count))

fruit_detector_fruit_count = 0 

strawbery_files = np.array(glob("/home/workspace/images/*/Strawberry/*"))
strawbery_files_short = strawbery_files[:100]

for path in strawbery_files_short:
    resultindex = fruit_detector(path)
    if resultindex == 949:
        fruit_detector_fruit_count+=1

print("Percent of strawbery images detected as a strawbery: %d" % (fruit_detector_fruit_count))

fruit_detector_fruit_count = 0 

orange_files = np.array(glob("/home/workspace/images/*/Orange/*"))
orange_files_short = orange_files[:100]

for path in orange_files_short:
    resultindex = fruit_detector(path)
    if resultindex == 950:
        fruit_detector_fruit_count+=1

print("Percent of orange images detected as a orange: %d" % (fruit_detector_fruit_count))


fruit_detector_fruit_count = 0 

lemon_files = np.array(glob("/home/workspace/images/*/Lemon/*"))
lemon_files_short = lemon_files[:100]

for path in lemon_files_short:
    resultindex = fruit_detector(path)
    if resultindex == 951:
        fruit_detector_fruit_count+=1

print("Percent of Lemon images detected as a Lemon: %d" % (fruit_detector_fruit_count))


Percent of apples images detected as an apple: 100
Percent of bananas images detected as a banana: 9
Percent of strawbery images detected as a strawbery: 100
Percent of orange images detected as a orange: 33
Percent of Lemon images detected as a Lemon: 10


---
## Create a CNN to Classify Fruits (from Scratch)


### (IMPLEMENTATION) Specify Data Loaders for the Fruits Dataset

To create data augmentation, we will apply transforms to the training set that will rotate the input images, flip them and apply some color adjustsments.  We will also normalize the input images and scale them all to 100x100 pixels.  This will improve our results as it will simulate more input for training and standarize scaling of images


In [4]:
normalize_transform = transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])

resize_transform = transforms.Resize(100)
centercrop_transform = transforms.CenterCrop(100)

data_transforms = {
    'Training': transforms.Compose([
        transforms.RandomResizedCrop(100),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        normalize_transform
    ]),
    'Validation': transforms.Compose([
        resize_transform,
        centercrop_transform,
        transforms.ToTensor(),
        normalize_transform
    ]), 
    'Test': transforms.Compose([
        resize_transform,
        centercrop_transform,
        transforms.ToTensor(),
        normalize_transform
    ]),    
}

### (IMPLEMENTATION) Loading the images from the "Test" and "Train" directories.  We will use 20% of the training data as a validation set

Create a CNN to classify fruit class.  Use the template in the code cell below.  Assumption is that images from Kaggle have been placed in the "images" folder and there is a "Training" and "Test" directory.  We will adjust our training set to use 20% for data validation.  Since our classes our imbalanced (some classes have more images than others), we will apply a WeightedSampler to avoid overfitting.

In [28]:

data_dir = os.getcwd() + '/images'

image_datasets_scratch = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                              for x in ['Training', 'Test']}


print("Training: %d" % (len(image_datasets_scratch['Training'])))
print("Testing %d" % (len(image_datasets_scratch['Test'])))

class_names = image_datasets_scratch['Training'].classes
class_count = len(class_names)
print(class_names)
print("Class count %d" % (class_count))
                      
batch_size = 20
num_workers = 0
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(image_datasets_scratch['Training'])
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
loaders_scratch = {}
loaders_scratch['Training'] = torch.utils.data.DataLoader(image_datasets_scratch['Training'], batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
loaders_scratch['Validation'] = torch.utils.data.DataLoader(image_datasets_scratch['Training'], batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
loaders_scratch['Test'] = torch.utils.data.DataLoader(image_datasets_scratch['Test'], batch_size=batch_size, 
    num_workers=num_workers)



Training: 57276
Testing 19548
['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Blueberry', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Chestnut', 'Clementine', 'Cocos', 'Dates', 'Ginger Root', 'Granadilla', 'Grape Blue', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grape White 3', 'Grape White 4', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Hazelnut', 'Huckleberry', 'Kaki', 'Kiwi', 'Kohlrabi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Lychee', 'Mandarine', 'Mango', 'Mango Red', 'Mangostan', 'Maracuja', 'Melon Piel de Sapo', 'Mulberry', 'Nectarine', 'Nectarine Flat', 'Nut Forest',

### (IMPLEMENTATION) Model Architecture

Create a CNN class that will classify our fruits. We will use 5 convolutional layers with max pooling layer after each. We will then put that through 2 fully connected layers.  We will use a Kernel of 3x3.

In [27]:

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        # convolutional layers
         # convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1 )       
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1 )
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1 )
        self.conv4 = nn.Conv2d(64, 128, 3, padding = 1 )
        self.conv5 = nn.Conv2d(128, 256, 3, padding = 1 )
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        
        self.conv_bn2 = nn.BatchNorm2d(16)
        self.conv_bn3 = nn.BatchNorm2d(32)
        self.conv_bn4 = nn.BatchNorm2d(64)
        self.conv_bn5 = nn.BatchNorm2d(128)
        self.conv_bn6 = nn.BatchNorm2d(256)
        
        self.fc1 = nn.Linear(256 * 3 * 3, 500)
        self.fc2 = nn.Linear(500, class_count)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.conv_bn2(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.conv_bn3(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.conv_bn4(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.conv_bn5(x)
        x = self.pool(F.relu(self.conv5(x)))
        x = self.conv_bn6(x)
        
        # flatten image input
        x = x.view(-1, 256 * 3 * 3)        
       
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# instantiate the CNN
model_scratch = Net()
print(model_scratch)
# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2)
  (conv_bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_feat

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [8]:

### TODO: select loss function. This will penalize incorrect probabilities
criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.0008)


### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at the given file input path.

In [7]:
total_step = len(loaders_scratch['Training'])
loss_list = []
acc_list = []

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    print('Training started')
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        print('Started epoch')
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['Training']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            data, target = data.to(device), target.to(device)
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
      
            optimizer.zero_grad()
            # Get output
            output = model(data)               
            # Calculate loss
            loss = criterion(output, target)
            loss.backward()
            optimizer.step() 
            train_loss = train_loss + (1 / (batch_idx + 1)) * (loss.data - train_loss)
            
            # Track the accuracy
            total = target.size(0)
            _, predicted = torch.max(output.data, 1)
            correct = (predicted == target).sum().item()
            acc_list.append(correct / total)

            if (batch_idx + 1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, n_epochs, batch_idx + 1, total_step, loss.item(),
                              (correct / total) * 100))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['Validation']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + (1 / (batch_idx + 1)) * (loss.data - valid_loss)
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Saving model: {} \tNew Valid Loss: {:.6f} \tPrevious Valid Loss: {:.6f}'.format(
                epoch, 
                valid_loss,
                valid_loss_min
                ))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss     
    # return trained model
    return model


### (IMPLEMENTATION) Train and Validate the Model

We will be training the model over 25 epochs and saving our models state to a local path.  We can see that the model validation loss is optimal around epoch 17 where the last model parameters get saved.  We will now use this model to run through our test function.  Our metric to optimize is log loss and we can see the best results are around 0.06 of loss.

In [11]:


# train the model
model_scratch = train(25, loaders_scratch, model_scratch, optimizer_scratch, criterion_scratch, use_cuda, 'model_fruit_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_fruit_scratch.pt'))

Epoch: 1 	Training Loss: 1.221388 	Validation Loss: 0.495568
Saving model: 1 	New Valid Loss: 0.495568 	Previous Valid Loss: inf
Epoch: 2 	Training Loss: 0.540813 	Validation Loss: 0.352751
Saving model: 2 	New Valid Loss: 0.352751 	Previous Valid Loss: 0.495568
Epoch: 3 	Training Loss: 0.410848 	Validation Loss: 0.408099
Epoch: 4 	Training Loss: 0.352851 	Validation Loss: 0.195776
Saving model: 4 	New Valid Loss: 0.195776 	Previous Valid Loss: 0.352751
Epoch: 5 	Training Loss: 0.305918 	Validation Loss: 0.254082
Epoch: 6 	Training Loss: 0.268454 	Validation Loss: 0.192788
Saving model: 6 	New Valid Loss: 0.192788 	Previous Valid Loss: 0.195776
Epoch: 7 	Training Loss: 0.252264 	Validation Loss: 0.111330
Saving model: 7 	New Valid Loss: 0.111330 	Previous Valid Loss: 0.192788
Epoch: 8 	Training Loss: 0.230439 	Validation Loss: 0.154823
Epoch: 9 	Training Loss: 0.216887 	Validation Loss: 0.105943
Saving model: 9 	New Valid Loss: 0.105943 	Previous Valid Loss: 0.111330
Epoch: 10 	Trainin

### (IMPLEMENTATION) Test model

We will be use the data set in the images/Test folder to test our newly created model.  We see that both accuracy and log loss looks to be in a good state.

In [16]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['Test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        #pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        #correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        #total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))


In [9]:

# call test function
model_scratch.load_state_dict(torch.load('model_fruit_scratch.pt'))
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Test Loss: 0.029805


Test Accuracy: 99% (19375/19548)


---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Fruit types (using Transfer Learning)

We attempted to use transfer learning as well, even though our results from model from scratch where good.  I did a few passes but every time the model took a very long time to train, which was one of the reasons that my GPU credits with Udacity workspace ran out.  I also tried the training process in sagemaker and that also took a very long time.  The model implemenation is outlined below as it had been outlined below.  I used resnet and VGG16 and they were both not succesful training processes.  I only replaced the last layer in the model in both instances.

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify fruit type.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [17]:
model_transfer = models.resnet50(pretrained=True)
for param in model_transfer.parameters():
    param.requires_grad = False

fc_inputs = model_transfer.fc.in_features
 
model_transfer.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 10), 
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)


if use_cuda:
    model_transfer.cuda()
print(model_transfer)


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/generic/THCTensorCopy.c:20

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [14]:
criterion_transfer = nn.CrossEntropyLoss()

optimizer_transfer = optim.Adam(model_scratch.parameters(), lr=0.001)

### (IMPLEMENTATION) Train and Validate the Model

We will have to convert the images to the same input size that pretrained models expect, 224x224 pixels.  We will save these new model parameters in a new model.  This process was able to start but made my GPU credits run out so i had to stop it.  We will use our custom CNN to test below.

In [15]:
# train the model
n_epochs = 20

normalize_transfer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

resize_transform = transforms.Resize(256)
centercrop_transform = transforms.CenterCrop(224)

data_transforms_transfer = {
    'Training': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        normalize_transfer
    ]),
    'Validation': transforms.Compose([
        resize_transform,
        centercrop_transform,
        transforms.ToTensor(),
        normalize_transfer
    ]), 
    'Test': transforms.Compose([
        resize_transform,
        centercrop_transform,
        transforms.ToTensor(),
        normalize_transfer
    ]),    
}
image_datasets_transfer = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms_transfer[x])
                              for x in ['Training', 'Test']}

# prepare data loaders
loaders_transfer = {}
loaders_transfer['Training'] = torch.utils.data.DataLoader(image_datasets_transfer['Training'], batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
loaders_transfer['Validation'] = torch.utils.data.DataLoader(image_datasets_transfer['Training'], batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
loaders_transfer['Test'] = torch.utils.data.DataLoader(image_datasets_transfer['Test'], batch_size=batch_size, 
    num_workers=num_workers)

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

model_transfer = torch.load('model_transfer.pt')

Training started
Started epoch


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/generated/../THCReduceAll.cuh:339

In [22]:
#Load a previously trained model settings
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of fruit images.

In [23]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

Test Loss: 0.328588


Test Accuracy: 89% (752/836)


### (IMPLEMENTATION) Predict Fruit type with the Model

Write a function that takes an image path as input and model parameter file and returns the fruit that is predicted by your model.  

In [48]:

def predict_fruit_transfer(img_path, model_path):
    # load the image and return the predicted breed

    resize_transform = transforms.Resize(100)
    centercrop_transform = transforms.CenterCrop(100)
    normalize_transform = transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
    
    transformation = transforms.Compose([
        resize_transform,   
        centercrop_transform,
        transforms.ToTensor(),
        normalize_transform
    ])
    
    image = Image.open(img_path)
     # Preprocess the image
    image_transform = transformation(image).float()

    # Add extra dimension to treat as batch
    image_transform = image_transform.unsqueeze_(0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if use_cuda:
        image = Variable(image_transform.cuda()) 
    else:
        image = Variable(image_transform.to(device))
    #Send to GPU if available
    image_transform = image_transform.to(device)  
    # Turn the input into a Variable
    image_model_input = Variable(image_transform)
    
    model = Net()
    # move tensors to GPU if CUDA is available
    if use_cuda:
        model.cuda()
    
    model_params = torch.load(model_path, map_location=torch.device(device))
    model.load_state_dict(model_params)

    #Prepare model
    model.eval()
    output = model(image_model_input)
    
    #Get values and index of max value
    values, predict_index = torch.max(output, 1)
    
    return predict_index, classes[predict_index]
    


In [49]:

import matplotlib.image as mpimg
from glob import glob
import random

def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    pred_index, fruit_class = predict_fruit_transfer(img_path, 'model_fruit_scratch.pt')
    
    # extract breed from image path
    actual_fruit = img_path.split("/")[-1].split('_')[:-1]
    actual_fruit = " ".join(actual_fruit)
    
    # display test image
    fig = plt.figure(figsize=(16,4))
    ax = fig.add_subplot(1,2,1)
    img = mpimg.imread(img_path)
    ax.imshow(img)
    
    # display sample of matching breed images
    base_path = os.getcwd() + '/images/Validation'
    fruit_directory =  '/'.join([base_path, str(fruit_class)])
    fruit_file = random.choice(os.listdir(fruit_directory))
    fruit_path = '/'.join([fruit_directory, fruit_file])
    
    ax = fig.add_subplot(1,2,2)
    img = mpimg.imread(fruit_path)
    ax.imshow(img.squeeze())
    
    fruit_class = fruit_class.split(".")[-1].replace("_", " ")
    plt.title(breed_class)
    plt.show()   
    
    print(f"Input fruit type: {actual_fruit}\n")
    print(f"Predicted fruit type: {fruit_class}\n")

run_app("banana.jpeg")

RuntimeError: Error(s) in loading state_dict for Net:
	size mismatch for conv1.weight: copying a param with shape torch.Size([16, 3, 5, 5]) from checkpoint, the shape in current model is torch.Size([16, 3, 3, 3]).

### (SageMaker) Model Training in SageMaker - Bonus due to GPU training running out

Initially, the tests that were run where done in the Udacity Workspace. All my code above had been ran and tweeked successfully but had a few more things to test. My GPU balance ran out but still had some tests to run, so i created a "source_pytorch" folder, where I have my training script.  I was successfully able to run my training script.  Assumption is that the image files from Kaggle have been downloaded and placed in the workspace.  You'll have to uncomment a line below where the files get uploaded to S3

In [51]:
#Required to start.  Place kaggle downloaded images into "images" folder that can be accessed by the Jupyter notebook
import boto3
import sagemaker
from PIL import ImageFile  
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True  

# your import and estimator code, here
from sagemaker.pytorch import PyTorch

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

# should be the name of directory you created to save your features data
data_dir = 'images'

# set prefix, a descriptive name for a directory  
prefix = 'fruit_images'

# upload all data to S3 - UNCOMMENT THIS LINE TO TEST
#input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_data = "s3://sagemaker-us-east-2-979781493470/fruit_images"
print(input_data)

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)
print(output_path)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'class_count': 114,
                        'epochs': 10, # could change to higher
                        'lr': 0.0008
                    })
estimator.fit({'train': input_data})

s3://sagemaker-us-east-2-979781493470/fruit_images
s3://sagemaker-us-east-2-979781493470/fruit_images
2019-08-07 04:37:42 Starting - Starting the training job...
2019-08-07 04:37:45 Starting - Launching requested ML instances......
2019-08-07 04:38:46 Starting - Preparing the instances for training...
2019-08-07 04:39:33 Downloading - Downloading input data..........................................
2019-08-07 04:46:42 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-07 04:46:57,252 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-07 04:46:57,254 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-08-07 04:46:57,267 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-07 04:46:57,268 sagemaker_pytorch_container.training INFO     Invoking user training script

In [ ]:
%%time

import boto3
import sagemaker
from PIL import ImageFile  
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True  

# your import and estimator code, here
from sagemaker.pytorch import PyTorch

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

# should be the name of directory you created to save your features data
data_dir = 'images'

# set prefix, a descriptive name for a directory  
prefix = 'fruit_images'

# upload all data to S3 - UNCOMMENT THIS LINE TO TEST
#input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_data = "s3://sagemaker-us-east-2-979781493470/fruit_images"
print(input_data)

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)
print(output_path)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'class_count': 114,
                        'epochs': 10, # could change to higher
                        'lr': 0.0008
                    })
estimator.fit({'train': input_data})

s3://sagemaker-us-east-2-979781493470/fruit_images
s3://sagemaker-us-east-2-979781493470/fruit_images
2019-08-07 06:45:02 Starting - Starting the training job...
2019-08-07 06:45:06 Starting - Launching requested ML instances......
2019-08-07 06:46:08 Starting - Preparing the instances for training...
2019-08-07 06:46:56 Downloading - Downloading input data.......................................
2019-08-07 06:53:31 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-07 06:53:46,557 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-07 06:53:46,560 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-08-07 06:53:46,573 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-07 06:53:49,591 sagemaker_pytorch_container.training INFO     Invoking user training script.
2

In [ ]:

%%time

# uncomment, if needed
from sagemaker.pytorch import PyTorchModel
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.0',
                     entry_point='predict.py',
                     source_dir='source_pytorch')

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')